In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
connectomes_mapping = pd.read_csv("./data/HCP-MMP1_RegionsCorticesList.csv")
connectomes_mapping = connectomes_mapping.drop(columns=["# ID", "Lobe"])
connectomes_mapping.head()

,Labelname,RegionLongName,LR,Cortex,Cortex_ID
0,L_V1,Primary_Visual_Cortex_L,L,Primary_Visual,1
1,L_MST,Medial_Superior_Temporal_Area_L,L,MT+_Complex_and_Neighboring_Visual_Areas,5
2,L_V6,Sixth_Visual_Area_L,L,Dorsal_Stream_Visual,3
3,L_V2,Second_Visual_Area_L,L,Early_Visual,2
4,L_V3,Third_Visual_Area_L,L,Early_Visual,2


In [3]:
def symmetrize_matrix(matrix):

    return (matrix + matrix.T) / 2

In [4]:
def reduce_connectome(connectome_matrix, connectome_mapping):
    assert connectome_matrix.shape[0] == connectome_matrix.shape[1]
    assert connectome_matrix.shape[0] == len(connectome_mapping)

    if isinstance(connectome_matrix, pd.DataFrame):
        connectome_matrix = connectome_matrix.values
    
    connectome_matrix = symmetrize_matrix(connectome_matrix)

    cortex_ids = connectome_mapping["Cortex_ID"].values

    reduced_matrix = np.zeros((27, 27))

    for i in range(1, 28): 
        for j in range(1, 28):  
            rows = np.where(cortex_ids == i)[0]
            cols = np.where(cortex_ids == j)[0]

            submatrix = connectome_matrix[np.ix_(rows, cols)]

            if submatrix.size > 0:  
                reduced_matrix[i - 1, j - 1] = np.mean(submatrix) 

    for i in range(27):
        for j in range(i + 1, 27):  
            reduced_matrix[j, i] = 0 

    return pd.DataFrame(
        reduced_matrix, 
        columns=[f"Cortex_{i}" for i in range(1, 28)], 
        index=[f"Cortex_{i}" for i in range(1, 28)]
    )


In [8]:
import zipfile
zip_file_path = "./data/Connectomes.zip"

extract_to = "./data"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


In [6]:
def process_connectomes_in_directory(connectome_dir, connectome_mapping, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    
    for file in os.listdir(connectome_dir):
        if file.endswith(".csv"):  
            file_path = os.path.join(connectome_dir, file)
            
            connectome_matrix = pd.read_csv(file_path, header=None)

            reduced_matrix = reduce_connectome(connectome_matrix, connectome_mapping)

            patient_id = file.split("-")[1].split("s")[1]

            output_file = os.path.join(output_dir, f"{patient_id}_reduced.csv")

            reduced_matrix.to_csv(output_file, index=True)



In [7]:
connectome_directories = [
    "./Connectomes/ses-T1",
    "./Connectomes/ses-T2",
    "./Connectomes/ses-T3",
    "./Connectomes/ses-T4"
]

output_base_directory = "./data/Reduced_Connectomes"

for connectome_directory in connectome_directories:
    session_name = connectome_directory.split('/')[-1] 
    output_directory = f"{output_base_directory}/{session_name}"
    
    process_connectomes_in_directory(connectome_directory, connectomes_mapping, output_directory)

    print(f"{connectome_directory} done -> {output_directory}")


./Connectomes/ses-T1 done -> ./data/Reduced_Connectomes/ses-T1
./Connectomes/ses-T2 done -> ./data/Reduced_Connectomes/ses-T2
./Connectomes/ses-T3 done -> ./data/Reduced_Connectomes/ses-T3
./Connectomes/ses-T4 done -> ./data/Reduced_Connectomes/ses-T4
